In [ ]:
import os
import sys
sys.path.insert(0, '/work/greenhouse-simulator-2/')

import math
import numpy as np
import pandas as pd

from crops.crop import Crop
from helpers.types import *
from helpers.data_prep import *
from helpers.visualization import *
from helpers.solar_conversions import *

In [ ]:
class SweetBasil(Crop):
    """
    Source of magic numbers: https://docs.google.com/spreadsheets/d/15W7doRV3CC-cBJzIBFd_GSMAgq3gd0UYVjLGZALZG58/edit#gid=0
    """
    def __init__(self, *args, **kwargs):
        # Define crop specific constants
        self.initial_weight: g = 2
        self.initial_leaf_area: cm2 = 40
        self.grow_period: d = 21
        self.photoperiod: h = 16
        self.target_DLI: mol_per_m2_day = 13

        super(SweetBasil, self).__init__(
            initial_weight=self.initial_weight, 
            initial_leaf_area=self.initial_leaf_area, 
            grow_period=self.grow_period, 
            target_DLI=self.target_DLI, 
            *args, 
            **kwargs
        )


    def _get_final_plant_props(self, dli: mol_per_m2_day) -> g:
        # Diminishing returns above DLI 17
        if dli > 17:
            dli = 17

        final_weight: g = 1.25 * dli + 2.35 # TODO: very rough linear approximation, come up with a better function
        final_leaf_area: cm2 = 25.4 * dli + 184 # TODO: very rough linear approximation, come up with a better function
        return final_weight, final_leaf_area


    def _get_growth_coeff_at(self, hour: h):
        return 0.0754 * math.exp(0.124 * (hour / 24))


    def _get_specific_photosynthetic_rate(self, dli: mol_per_m2_day) -> umol_per_m2_s:
        # Diminishing returns above DLI 17
        if dli > 17:
            dli = 17

        return 0.706 * dli + 0.115


    def _get_specific_transpiration_rate(self, dli: mol_per_m2_day) -> mmol_per_m2_s:
        # Diminishing returns above DLI 17
        if dli > 17:
            dli = 17
            
        return 0.101 * dli + 0.472

In [ ]:
sweet_basil = SweetBasil(plants_per_barrel=200, barrel_count=4, time_period=3600)

### Crop specific values over time

In [ ]:
if not os.environ.get("USING_RUN"):
    resample_period = "60min"

    sweet_basil = SweetBasil(plants_per_barrel=200, barrel_count=4, time_period=pd.to_timedelta(resample_period).total_seconds())
    df = get_weather_data(date_from="2020-07-02", date_to="2020-07-23", resample_period=resample_period)

    total_CO2_assimilated: mol = 0
    total_H2O_evaporated: mol = 0
    for i, row in df.iterrows():
        # Calculate projected Daily Light Integral (DLI) from irradiance
        irradiance: W_per_m2 = row["solarradiation"]
        PPFD: umol_per_m2_s = irradiance_to_PPFD(irradiance)
        dli: mol_per_m2_day = PPFD_to_projected_DLI(PPFD, sweet_basil.photoperiod)

        # Call `grow` function of tested crop
        CO2_assimilation_rate, CO2_assimilated, H20_evaporation_rate, H2O_evaporated = sweet_basil.grow(dli)

        # Register assimilation/evaporation rates
        df.loc[i, "CO2_assimilation_rate"] = CO2_assimilation_rate
        df.loc[i, "H20_evaporation_rate"] = H20_evaporation_rate

        # Register assimilation/evaproation totals
        total_CO2_assimilated += CO2_assimilated
        total_H2O_evaporated += H2O_evaporated
        df.loc[i, "CO2_assimilated"] = total_CO2_assimilated
        df.loc[i, "H2O_evaporated"] = total_H2O_evaporated

        # Register growth progress
        df.loc[i, "total_crop_weight"] = sweet_basil.weights[-1].sum()

        # Register weight and leaf_are of 5 evenly spaced out plant
        for j in range(5):
            df.loc[i, f"crop_{j*40}_weight"] = sweet_basil.weights[-1][j*40]

        for j in range(5):
            df.loc[i, f"crop_{j*40}_leaf_area"] = sweet_basil.leaf_areas[-1][j*40]



In [ ]:
plots = None
if not os.environ.get("USING_RUN"):
    multiline_cols = [f"crop_{j*40}_weight" for j in range(5)]
    plots = plot_multiline(df, multiline_cols, width=900, height=235, title="Weights of 5 plants over time", y_label="g / plant", legend_label="Weights", date_format="")
plots

In [ ]:
plots = None
if not os.environ.get("USING_RUN"):
    multiline_cols = [f"crop_{j*40}_leaf_area" for j in range(5)]
    plots = plot_multiline(df, multiline_cols, width=900, height=235, title="Leaf areas of 5 plants over time", y_label="cm2 / plant", legend_label="Leaf areas", date_format="")
plots

In [ ]:
if not os.environ.get("USING_RUN"):
    print(f"Harvested weight: {round(sweet_basil.harvested_weight) / 1000} kg")

In [ ]:
plots = None
if not os.environ.get("USING_RUN"):
    plots = plot_timeline(df, "total_crop_weight", width=900, height=235, y_label="g crop")
plots

In [ ]:
plots = None
if not os.environ.get("USING_RUN"):
    plots = plot_timeline(df, "CO2_assimilated", width=450, height=235, y_label="mol CO2") | \
    plot_timeline(df, "H2O_evaporated", width=450, height=235, y_label="mol H2O")
plots

In [ ]:
plots = None
if not os.environ.get("USING_RUN"):
    plots = plot_timeline(df, "CO2_assimilation_rate", width=450, height=235, y_label="mol CO2 / s") | \
    plot_timeline(df, "H20_evaporation_rate", width=450, height=235, y_label="mol H2O / s")
plots

### Values at harvest vs. DLI

In [ ]:
plots = None
if not os.environ.get("USING_RUN"):
    dli_vs_props = []
    for dli in range(25):
        weight, leaf_area = sweet_basil._get_final_plant_props(dli)
        specific_photosynthetic_rate: umol_per_m2_s = sweet_basil._get_specific_photosynthetic_rate(dli)
        specific_transpiration_rate: mmol_per_m2_s = sweet_basil._get_specific_transpiration_rate(dli)


        dli_vs_props.append({
            "dli": dli,
            "weight": weight,
            "leaf_area": leaf_area,
            "specific_photosynthetic_rate": specific_photosynthetic_rate,
            "specific_transpiration_rate": specific_transpiration_rate
        })

    df = pd.DataFrame(dli_vs_props)

    plots = plot_x_y_multiline(df, "dli", ["weight"], x_label="DLI", y_label="weight (g)", legend_label="", title="DLI vs. final weight") | \
    plot_x_y_multiline(df, "dli", ["leaf_area"], x_label="DLI", y_label="leaf area (cm2)", legend_label="", title="DLI vs. final leaf area")
plots


In [ ]:
plots = None
if not os.environ.get("USING_RUN"):
    plots = plot_x_y_multiline(df, "dli", ["specific_photosynthetic_rate"], x_label="DLI", y_label="photosynthetic rate (umol / m2 s)", legend_label="", title="DLI vs. photosynthetic rate") | \
    plot_x_y_multiline(df, "dli", ["specific_transpiration_rate"], x_label="DLI", y_label="transpiration rate (mmol / m2 s)", legend_label="", title="DLI vs. transpiration rate")
plots

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>